# Imports

In [ ]:
from pathlib import Path
import numpy as np
import torch
import math
from tqdm import tqdm
from torchvision.datasets import CocoDetection
import matplotlib.pyplot as plt
import torchvision

In [ ]:
from abbyy_course_cvdl_t3.coco_text import COCO_Text
from abbyy_course_cvdl_t3 import coco_evaluation
from abbyy_course_cvdl_t3.utils import evaluate_ap_from_cocotext_json
from abbyy_course_cvdl_t3.utils import dump_detections_to_cocotext_json

In [ ]:
cd ../task2

In [ ]:
import abbyy_course_cvdl_t2
from abbyy_course_cvdl_t2.convert import PointsToObjects, ObjectsToPoints
from abbyy_course_cvdl_t2.impl.train import train
from abbyy_course_cvdl_t2.impl.data import CocoTextDetection, CocoDetectionPrepareTransform
from abbyy_course_cvdl_t2.network import CenterNet
from abbyy_course_cvdl_t2.loss import CenterNetLoss

In [ ]:
cd ../task3

In [ ]:
device = torch.device('cuda:0')

# Data

In [ ]:
base = Path("/home/alexander/Downloads/coco2014/")
anno_path = base / 'cocotext.v2.json'
images_path = base / 'images/train2014'

assert anno_path.exists(), "Set your own path to annotation"
assert images_path.exists(), "Set your own path to images"


In [ ]:
ct = COCO_Text(anno_path)

In [ ]:
def draw_boxes(image_np, yc_t, xc_t, hy_t, wx_t, color=(0, 255, 0), thick=1):
    img = np.array(image_np)
    num_boxes = len(yc_t)
    val = np.array(color)
    for idx in range(num_boxes):
        yc = yc_t[idx]
        xc = xc_t[idx]
        hy = hy_t[idx]
        wx = wx_t[idx]
        img[
                yc - hy//2 - thick : yc - hy//2 + thick, 
                xc - wx//2 : xc + wx//2, 
        ] = val
        img[
                yc + hy//2 - thick : yc + hy//2 + thick, 
                xc - wx//2 : xc + wx//2, 
        ] = val

        img[
                yc - hy//2 : yc + hy//2, 
                (xc - wx//2 - thick): (xc - wx//2 + thick), 
        ] = val
        img[
                yc - hy//2 : yc + hy//2, 
                xc + wx//2 - thick: xc + wx//2 + thick, 
        ] = val
    return img
        
        

In [ ]:
n_classes = 2
input_shape = (256, 256)

In [ ]:
ds_train = CocoTextDetection(
    Path("/home/alexander/Downloads/coco2014/images/train2014"),
    Path("/home/alexander/Downloads/coco2014/cocotext.v2.json"),
    transforms=CocoDetectionPrepareTransform(size=input_shape, #ids_map = {1: 0, 0: 0}
),
    area_fraction_threshold=1/32/32
)

In [ ]:
ds_val = CocoTextDetection(
    images_path,
    Path(anno_path),
    transforms=CocoDetectionPrepareTransform(size=input_shape, #ids_map = {1: 0, 0: 0}
    ),
    area_fraction_threshold=1/32/32,
    split='val'
)

In [ ]:
x, y = ds_train[6]

In [ ]:
y[:10]

In [ ]:
trainloader = torch.utils.data.DataLoader(
    ds_train, batch_size=2, shuffle=True
)

In [ ]:
valloader = torch.utils.data.DataLoader(
    ds_val, batch_size=1, shuffle=False
)

# Learning

In [ ]:
net = CenterNet(head_kwargs={'c_classes': n_classes}, nms_kwargs={'kernel_size': 5})
crit = CenterNetLoss(obj_to_points=ObjectsToPoints(num_classes=n_classes))#, l_size_lambda=3)

In [ ]:
net = train(ds_train, net=net, criterion=crit, batch_size=64, epochs=2, device=torch.device(device), lr=1e-3)

In [ ]:
torch.save(net, 'centernet.pth')

# Quality

## Postprocessing and visualization

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
#net = torch.load('centernet.pth')

In [ ]:
net.to(device)
net.eval()

In [ ]:
x, y = ds_train[6]

with torch.no_grad():
    yp_heat = net(x[None].to('cuda:0')).cpu()    
    yp = net(x[None].to('cuda:0'), return_objects=True).cpu()

In [ ]:
print('class 0 heatmap')
plt.matshow(yp_heat[0][0])
plt.show()

print('class 1 heatmap')
plt.matshow(yp_heat[0][1])
plt.show()

print('w_heatmap')
plt.matshow(yp_heat[0][-1])
plt.show()

print('h heatmap')
plt.matshow(yp_heat[0][-2])
plt.show()

In [ ]:
def draw_boxes(image_np, yc_t, xc_t, hy_t, wx_t, color=(0, 255, 0), thick=1):
    img = np.array(image_np)
    num_boxes = len(yc_t)
    val = np.array(color)
    for idx in range(num_boxes):
        yc = yc_t[idx]
        xc = xc_t[idx]
        hy = hy_t[idx]
        wx = wx_t[idx]

        hyd2 = torch.div(hy, 2, rounding_mode='trunc')
        wxd2 = torch.div(wx, 2, rounding_mode='trunc')

        img[
                yc - hyd2 - thick : yc - hyd2 + thick, 
                xc - wxd2 : xc + wxd2, 
        ] = val
        img[
                yc + hyd2 - thick : yc + hyd2 + thick, 
                xc - wxd2 : xc + wxd2, 
        ] = val

        img[
                yc - hyd2 : yc + hyd2, 
                (xc - wxd2 - thick): (xc - wxd2 + thick), 
        ] = val
        img[
                yc - hyd2 : yc + hyd2, 
                xc + wxd2 - thick: xc + wxd2 + thick, 
        ] = val
    return img


rnd = lambda x: torch.round(x).long()

plt.imshow(draw_boxes(
    x.permute(1, 2, 0).numpy(), rnd(yp[0, :3, 0]), rnd(yp[0, :3, 1]), rnd(yp[0, :3, 2]), rnd(yp[0, :3, 3]) 
))

In [ ]:
plt.imshow(draw_boxes(
    x.permute(1, 2, 0).numpy(), rnd(y[:, 0]), rnd(y[:, 1]), rnd(y[:, 2]), rnd(y[:, 3]) 
))

In [ ]:
yp

In [ ]:
y

## Saving to json

In [ ]:
#net = torch.load('centernet.pth')

def postprocess(data, target_shape, input_shape, min_conf=0.1):
    unf = {}
    res = data[data[:, -1] > min_conf]
    boxes = res[:, :4].copy()
    b = boxes.copy()
    boxes[:, 0] = (boxes[:, 0] - b[:, 2] / 2) * (target_shape[0] / input_shape[0])
    boxes[:, 1] = (boxes[:, 1] - b[:, 3] / 2) * (target_shape[1] / input_shape[1])
    boxes[:, 2] = boxes[:, 2] * (target_shape[0] / input_shape[0])
    boxes[:, 3] = boxes[:, 3] * (target_shape[1] / input_shape[1])
    unf['boxes'] = boxes
    unf['scores'] = res[:, -2]
    return unf


def extract_results(min_conf=0.1):
    prepared_preds = []
    img_ids = []

    net.eval()

    for num, img_id in enumerate(tqdm(ds_val.ids)):
        img_id = int(img_id)
        img_meta = ct.loadImgs(ids=[img_id])[0]

        with torch.no_grad():
            x = ds_val[num][0]
            
            pred = net(
                x[None, ...].to(device),
                return_objects=True
            )[0].detach().cpu().numpy()
            prepared_preds.append(
                postprocess(pred, target_shape=(img_meta['height'], img_meta['width']), input_shape=input_shape, min_conf=min_conf)
            )
            img_ids.append(img_id)

    scores = np.concatenate([u['scores'] for u in prepared_preds], axis=0)
    boxes = np.concatenate([u['boxes'] for u in prepared_preds], axis=0)

    image_ids = []
    for num, i in enumerate(img_ids):
        image_ids += [i] * len(prepared_preds[num]['boxes'])
    image_ids = np.array(image_ids)

    #print(len(image_ids.tolist()),
    #    len(boxes[:, 0].tolist()),
    #    len(boxes[:, 1].tolist()),
    #    len(boxes[:, 2].tolist()),
    #    len(boxes[:, 3].tolist()),
    #    len(scores.tolist()))

    dump_detections_to_cocotext_json(
        image_ids = image_ids.tolist(),
        xlefts=boxes[:, 0].tolist(),
        ytops=boxes[:, 1].tolist(),
        widths=boxes[:, 2].tolist(),
        heights=boxes[:, 3].tolist(),
        scores=scores.tolist(),
        path=f'predictions_{min_conf}.json'
    )
    
    ap, prec, rec = evaluate_ap_from_cocotext_json(
    coco_text=ct,
    path=f'predictions_{min_conf}.json'
    )
    return (ap, prec, rec, min_conf, len(scores))

In [ ]:
min_conf=0.1
ap, prec, rec, conf, n_obj = extract_results(min_conf=min_conf)

print(f"Итоговый скор AP на val для min_conf={min_conf}: {ap}")

plt.plot(prec, rec)
plt.xlabel('precision')
plt.ylabel('recall')
plt.title('PR curve')
plt.grid()

# Best AP

In [ ]:
aps = []

for min_conf in np.linspace(0.1, 0.3, 5):
    ap, prec, rec, conf, n_obj = extract_results(min_conf=min_conf)

    aps.append((ap, min_conf))

    print(n_obj)
    print(f"Итоговый скор AP на val для min_conf={min_conf}: {ap}")
    plt.plot(prec, rec)
    plt.xlabel('precision')
    plt.ylabel('recall')
    plt.title('PR curve')
    plt.grid()
    plt.show()

In [ ]:
best_values = sorted(aps, key=lambda x: x[0])[-1]

In [ ]:
best_values